In [1]:
import sys, os, time, pickle
from timeit import default_timer as timer
from humanfriendly import format_timespan

In [2]:
import pandas as pd
import numpy as np

In [3]:
from dotenv import load_dotenv
load_dotenv('admin.env')

True

In [4]:
from h1theswan_utils.microsoft_academic_api import EvaluateQuery

In [5]:
# this is the data for the fortunato review on Community Detection in Graphs
start = timer()
test_papers_df = pd.read_pickle('data/collect_haystack_20180409_2/test_papers.pickle')
target_papers_df = pd.read_pickle('data/collect_haystack_20180409_2/target_papers.pickle')
train_papers_df = pd.read_pickle('data/collect_haystack_20180409_2/train_papers.pickle')
print(format_timespan(timer()-start))

1.2 second


In [6]:
with open('data/collect_haystack_20180409_2/counter.pickle', 'rb') as f:
    c = pickle.load(f)

In [7]:
train_pids = train_papers_df.Paper_ID

In [25]:
# pid = train_pids.iloc[1]
pid = 2117526408
eq = EvaluateQuery("Id={}".format(pid))
eq.attributes = 'E'
r = eq.get()

In [26]:
import json
ent = r['entities'][0]
E = ent['E']
E = json.loads(E)
iabs = E['IA']['InvertedIndex']


In [27]:
# method 1
start = timer()
abs_words_1 = []
for i in range(E['IA']['IndexLength']):
    for k, v in iabs.items():
        if i in v:
            abs_words_1.append(k)
            break
print(timer()-start)

0.0010003820061683655


In [34]:
# method 2
start = timer()
iabs_rev = {}
abs_words_2 = []
for k, v in iabs.items():
    for idx in v:
        iabs_rev[idx] = k
for i in range(E['IA']['IndexLength']):
    try:
        abs_words_2.append(iabs_rev[i])
    except KeyError:
        pass
print(timer()-start)

0.0002965778112411499


In [12]:
from h1theswan_utils.microsoft_academic_api import convert_inverted_abstract_to_abstract_words
start = timer()
abs_words_3 = convert_inverted_abstract_to_abstract_words(iabs, index_length=E['IA']['IndexLength'])
print(timer()-start)

0.00010903365910053253


In [13]:
from h1theswan_utils.microsoft_academic_api import convert_inverted_abstract_to_abstract_words
start = timer()
abs_words_4 = convert_inverted_abstract_to_abstract_words(iabs)
print(timer()-start)

0.00010813958942890167


In [24]:
abs_words_3 == abs_words_4

True

In [14]:
indices = set()
for k, v in iabs.items():
    for idx in v:
        indices.add(idx)
max(indices)

81

In [35]:
abs_words_1 == abs_words_2

True

In [16]:
# method 2 is faster

In [29]:
" ".join(abs_words_1)

'The recent boom of large-scale online social networks (OSNs) both enables and necessitates the use of parallelizable and scalable computational techniques for their analysis. We examine the problem of real-time community detection and a recently proposed linear time―O(m) on a network with m edges―label propagation, or "epidemic" community detection algorithm. We identify characteristics and drawbacks of the algorithm and extend it by incorporating different heuristics to facilitate reliable and multifunctional real-time community detection. With limited computational resources, we employ the algorithm on OSN data with 1 × 10 6 nodes and about 58 X 10 6 directed edges. Experiments and benchmarks reveal that the extended algorithm is not only faster but its community detection accuracy compares favorably over popular modularity-gain optimization algorithms known to suffer from their resolution limits.'

In [37]:
len(test_papers_df)

2613341

In [39]:
test_papers_df.Paper_ID.to_csv('data/collect_haystack_20180409_2/test_paper_ids.txt', index=False)